# Cleanup
Konvertering af brevene i json-format til en enkelt csv-fil.

Vi renser ud i de mest basale tekstformateringer, såsom linjeskift, tabuleringer og dobbelt mellemrum.

Omdøber kolonnerne og konverterer datoerne til datetime-format og introducerer et id.


In [1]:
# load jsonfile from disk
import json
import pandas as pd
import sys
sys.path.insert(0, '../scripts')


C:\Users\christian.dalager\AppData\Local\Temp\ipykernel_27780\1269626062.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('../data/letters.json', 'r') as f:
    letters = json.load(f)

from MLStripper import strip_tags
# strip html tags from text

for letter in letters:
    letter['Text'] = strip_tags(letter['Text'])

df = pd.DataFrame(letters)

# lowercase all column names
df.columns = map(str.lower, df.columns)


In [3]:
# Text cleanup
df['text'] = df['text'].str.strip()

# replace crlf with lf
df['text'] = df['text'].str.replace('\r\n', '\n')
# replace leading and trailing whitespace
df['text'] = df['text'].str.replace('\n( )*\n', '\n\n', regex=True)

# replace multiple newlines with single newline
df['text'] = df['text'].str.replace('\n{3,}', '\n\n', regex=True)

# replace all occurrences double newlines with break markerr
df['text'] = df['text'].str.replace('\n\n', '<PARA>')

df['text'] = df['text'].str.replace('- - - -', '[ulæseligt]')
df['text'] = df['text'].str.replace('----', '[ulæseligt]')
df['text'] = df['text'].str.replace('- - -', '[ulæseligt]')
df['text'] = df['text'].str.replace('- -', '-')
df['text'] = df['text'].str.replace('--', '-')

# replace single newlines with space
df['text'] = df['text'].str.replace('\n', ' ')

# replace \xa0 with space
df['text'] = df['text'].str.replace('\xa0', ' ')
df['text'] = df['text'].str.replace('&lt;', ' ')

# replace \xa with ''
df['text'] = df['text'].str.replace('\xad', '')
df['text'] = df['text'].str.replace('\x95', '')

# replace multiple spaces with single space
df['text'] = df['text'].str.replace('\s+', ' ', regex=True)
df['text'] = df['text'].str.strip()


In [4]:
# Korrektur af fejl i data
df['text'] = df['text'].str.replace(' cg ', ' og ')

In [19]:
# find ocurrences of single letter words in a row


In [5]:
# Dates
df['date'] = pd.to_datetime(df['letterdate'])
df['date'] = df['date'].dt.date
df.drop(['letterdate'], axis=1, inplace=True)
df.sort_values(by=['date'], inplace=True)

In [6]:
# Add an id column as the first column
df.insert(0, 'id', range(1, 1 + len(df)))

# reorder columns
df = df[['id', 'date', 'sender', 'recipient','place','location', 'text']]

df.to_csv('../data/letters.csv', index=False)